In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
from pathlib import Path
import zipfile
import duckdb
import polars as pl

DATA_DIR = Path(os.getcwd()).parent / "data"

## First time setup
Download and unzip the data to the data directory.

In [16]:
!kaggle competitions download -c playground-series-s4e10 -p ../data

... resuming from 0 bytes (1515882 bytes left) ...
 69%|██████████████████████████▎           | 1.00M/1.45M [00:00<00:00, 3.01MB/s]
100%|██████████████████████████████████████| 1.45M/1.45M [00:00<00:00, 3.93MB/s]


In [30]:
with zipfile.ZipFile(DATA_DIR / "playground-series-s4e10.zip", 'r') as zipped:
    zipped.extractall(DATA_DIR)
os.listdir(DATA_DIR)

['train.csv',
 'sample_submission.csv',
 'test.csv',
 'playground-series-s4e10.zip']

## Constants

In [31]:
target = "loan_status"

## Exploratory Data analysis

In [29]:
train = pl.read_csv(DATA_DIR / "train.csv")
test = pl.read_csv(DATA_DIR / "test.csv")
train

id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
i64,i64,i64,str,f64,str,str,i64,f64,f64,str,i64,i64
0,37,35000,"""RENT""",0.0,"""EDUCATION""","""B""",6000,11.49,0.17,"""N""",14,0
1,22,56000,"""OWN""",6.0,"""MEDICAL""","""C""",4000,13.35,0.07,"""N""",2,0
2,29,28800,"""OWN""",8.0,"""PERSONAL""","""A""",6000,8.9,0.21,"""N""",10,0
3,30,70000,"""RENT""",14.0,"""VENTURE""","""B""",12000,11.11,0.17,"""N""",5,0
4,22,60000,"""RENT""",2.0,"""MEDICAL""","""A""",6000,6.92,0.1,"""N""",3,0
…,…,…,…,…,…,…,…,…,…,…,…,…
58640,34,120000,"""MORTGAGE""",5.0,"""EDUCATION""","""D""",25000,15.95,0.21,"""Y""",10,0
58641,28,28800,"""RENT""",0.0,"""MEDICAL""","""C""",10000,12.73,0.35,"""N""",8,1
58642,23,44000,"""RENT""",7.0,"""EDUCATION""","""D""",6800,16.0,0.15,"""N""",2,1
